In [ ]:
#Duffing[0,2000] by CNS

In [ ]:
function accuracy()
    ac=800;
    return ac;
end
setprecision(accuracy());

In [ ]:
function order()
    M=80;#60
    return M+1;
end

In [ ]:
function steps()
    h = BigFloat(1)/BigFloat(100)
    return h;
end

In [ ]:
function JCh(k,i)
     
    k = BigFloat(k);
    i = BigFloat(i);
    s = BigFloat(1);
    if i < k/2
        i = k - i;
    end
    for j in range(k,i+1,step=-1)
        
        j1=BigFloat(j)
        s = s*j1/(j1-i)
        
    end
    
    return s
end

In [ ]:
function DJCh()
    Cki = Array{BigFloat}(undef,order()-1,order()-1);
    for k in 1:order()-1
        for j in 1:k
            Cki[k,j] = 1
        end
    end
    
    for k in 2:order()-2
        for i in 1:k-1
           Cki[k+1,i+1] = Cki[k,i] + Cki[k,i+1]
        end
    end
    return Cki
end

In [ ]:
function multi()
    S = Array{BigFloat}(undef,1,order());
    S[1] = BigFloat(1)
    
    for i in 1:order()-1
        S[i+1] = S[i]*steps()/BigFloat(i)
    end
    
    return S
end

In [ ]:
fdot(x,y,z) = @. x*y*z

In [ ]:
function Sumxx(x,k,Cki)
    xx = Array{BigFloat}(undef,k)
    for i in 1:k
        xx[i] = sum(fdot(Cki[i,1:i],x[1:i],reverse!(x[1:i])))
    end
    return xx
end 

In [ ]:
function Sumxyz(x,k,Cki)
    return sum(fdot(Cki[k,1:k],Sumxx(x[1:end],k,Cki),reverse!(x[1:end])))
end 

In [ ]:
function Talor(x,y,z,Cki,Tin)
    
    for k in 1:order()-1
        if mod(k,4) == 1
            x[k+1] = y[k]
            y[k+1] = a*c^(k-1)*cos(c*Tin) - b*y[k] - x[k] - Sumxyz(x[1:k],k,Cki)
            z[k+1] = 0.0
        elseif mod(k,4) == 2
            x[k+1] = y[k]
            y[k+1] = -a*c^(k-1)*sin(c*Tin) - b*y[k] - x[k] - Sumxyz(x[1:k],k,Cki)
            z[k+1] = 0.0
        elseif mod(k,4) == 3
            x[k+1] = y[k]
            y[k+1] = -a*c^(k-1)*cos(c*Tin) - b*y[k] - x[k] - Sumxyz(x[1:k],k,Cki)
            z[k+1] = 0.0
        else
            x[k+1] = y[k]
            y[k+1] = a*c^(k-1)*sin(c*Tin) - b*y[k] - x[k] - Sumxyz(x[1:k],k,Cki)
            z[k+1] = 0.0
        end
    end
    
    return x,y,z
end

In [ ]:
function Iteration(x,y,z,Cki,Tin)
    
    for st in 1:Times
    
        x,y,z = Talor(x,y,z,Cki,Tin)
        
        x[1] = sum(x[1:end].*Dmulti[1:end])
        y[1] = sum(y[1:end].*Dmulti[1:end])
        z[1] = sum(z[1:end].*Dmulti[1:end])
    
        Lorenz[st,:] = [x[1],y[1],z[1]] 
        Lorenzf[st,:] = convert.(Float64,Lorenz[st,:])
        
        Tin = Tin + steps()
        if mod(st,1000) == 0
            println(st,") ","time=",convert(Float16,Tin),": ",Lorenzf[st,1:2])
        end
        
    end
    
    return Lorenz
    
end

In [ ]:
Times = 100*2000;      

a =  BigFloat(92) + BigFloat(825)/BigFloat(1000);
b =  BigFloat(1)/BigFloat(10);
c =  BigFloat(1)/BigFloat(1);

x = Array{BigFloat}(undef,order())
y = Array{BigFloat}(undef,order())
z = Array{BigFloat}(undef,order())

Lorenz = Array{BigFloat}(undef,Times,3);
HH = Array{BigFloat}(undef,Times,3);
Ep = Array{BigFloat}(undef,Times,1);
Lorenzf = Array{Float32}(undef,Times,3);
Dmulti = Array{BigFloat}(undef,1,order());
Cki = Array{BigFloat}(undef,order()-1,order()-1);

x[1] = BigFloat(0); 
y[1] = BigFloat(0); 
z[1] = BigFloat(0);
Tin = BigFloat(0)

Cki = DJCh();
Dmulti = multi();

In [ ]:
HH = Iteration(x,y,z,Cki,Tin);

In [ ]:
using Plots; 
plot(HH[1:end,1],HH[1:end,2])

In [ ]:
#write
function Wfile(AT,BE,Times)
#change name--------------------------------------------
    XLSX.openxlsx("CNS_Duffing.xlsx", mode="w") do xf
#-------------------------------------------------------
        sheet = xf[1]
        XLSX.rename!(sheet, "new_sheet")
#change location----------------------------------------
        sheet["A1"] = "T"
        sheet["B1"] = "X"
        sheet["A2",dim=1] = AT[1:end]
        sheet["B2",dim=1] = BE[1:end]
#-------------------------------------------------------
    end
end
#-----------------------------------------------------------------------------
using XLSX
#time-----------------------------------------------------
AT=Array{Float64}(undef,Times)
for i in 1:Times
    AT[i] = i*steps()
end
#time-----------------------------------------------------
BE = HH[:,1]
#--------------------------------------------------------
Wfile(AT,BE,Times);

In [ ]:
plot(xlabel = "x", ylabel = "x'")
clr = palette(:gist_rainbow)
Plots.plot!(HH[1:15000,1],HH[1:15000,2],
    linealpha = 1,
    linewidth = 1.5,
    linecolor = clr[1:end],
    legend = false
)

In [ ]:
plot(1:10000,HH[1:10000,1])